In [1]:
#!/usr/bin/env acii
# with validation set from Training set
import sys
print(sys.executable)

# From Stackoverflow
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Restrict TensorFlow to only use the fourth GPU
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')

        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)
        
import os
os.environ['PYTHONHASHSEED'] = '0'

from numpy.random import seed
from numpy import array, vstack, hstack, stack
from utils import unison_shuffled_copies, unison_shuffled_copies_two, NDStandardScaler
from utils import mk_dirs, create_csv, create_dirs, f1_m, precision_m, recall_m, create_multicsv
from utils import *

import os
import numpy as np
import pandas as pd
import seaborn as sns
from numpy.random import seed
from numpy import array, vstack, hstack, stack
import random as rn
rn.seed(4)
import gc
import datetime
import inspect
import pickle
import scipy.io
from scipy import stats
import scipy.signal as scisig
from scipy.stats import zscore
import matplotlib.pyplot as plt

from mega_model import *
from mega_model_resnet import *
from tensorflow.keras import backend as K

from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.utils import to_categorical
import tensorflow_addons as tfa
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

from sklearn.preprocessing import StandardScaler, label_binarize
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.utils import class_weight
import h5py
import neurokit2 as nk
from statistics import mean, mode, StatisticsError
from sklearn.preprocessing import MinMaxScaler

import warnings
warnings.filterwarnings('ignore')


seed(2)
tf.random.set_seed(42)
print(tf.keras.__version__)


main_path = r"X:\Thesis\matb2\Processed_Data"

with open(os.path.join(main_path, 'cola_ecg.pickle'), 'rb') as handle:
    sub_dict_ecg = pickle.load(handle)

with open(os.path.join(main_path, 'cola_labels.pickle'), 'rb') as handle:
    sub_label_ecg = pickle.load(handle)
    
with open(os.path.join(main_path, 'cola_eda.pickle'), 'rb') as handle:
    sub_dict_eda = pickle.load(handle)

c:\Users\Anubhav\anaconda3\envs\acii\python.exe
1 Physical GPUs, 1 Logical GPUs


c:\Users\Anubhav\anaconda3\envs\acii\lib\site-packages\tensorflow\python\keras\backend.py:434: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '


2.4.0


In [64]:
num_classes = 2
opt = Adam(learning_rate = 0.001)
model = mega_model(input_shape=[(2560, 1), (2560, 3)], attx_type='III', attx_st='all', classes = num_classes)

model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['acc', tfa.metrics.F1Score(num_classes=num_classes, threshold=0.5, average = 'macro')])

print(model.summary())

method = 'LOSO'
dataset_name = 'cola'

attx_type = ['II']
attx_st = ['two_three', 'all']

# attx_type = ['III']
# attx_st = ['all']

for conn_type in attx_type:
    
    for conn_stage in attx_st:
        print("--------------------------------------------------------------------------")
        print("Training for Type {}, Stage {}".format(conn_type, conn_stage))
        print("--------------------------------------------------------------------------\n")        
        
        hs, preds, clr = {}, {}, {}

        path_logs = r'X:/Data Files/TAFFC/Cola/'
        tensorbrd_dir, model_report, model_data, model_score, model_arch, model_fid, model_weights, model_files = create_dirs(path_logs)

        for i in sub_dict_ecg.keys():
            opt = tf.keras.optimizers.Adadelta(learning_rate = 0.001, rho=0.95)
            tb = tensorflow.keras.callbacks.TensorBoard(log_dir = os.path.join(tensorbrd_dir,
                                                                               datetime.datetime.now().strftime("%Y%m%d-%H%M%S")))

            X_test_ecg = sub_dict_ecg[i]
            y_test = sub_label_ecg[i]
            X_test_eda = sub_dict_eda[i]

            X_test_ecg = vstack(X_test_ecg)
            X_test_eda = vstack(X_test_eda)
            y_test = [x for z in y_test for x in z]


            X_ecg = [vstack(v) for k, v in sub_dict_ecg.items() if k != i]
            X_eda = [vstack(v) for k, v in sub_dict_eda.items() if k != i]
            y_train = [hstack(np.asarray(v)) for k, v in sub_label_ecg.items() if k != i]

            X_ecg = vstack(X_ecg)
            X_eda = vstack(X_eda)
            y_train = hstack(np.asarray(y_train))

            y_train = [1 if x > 5 else 0 for x in y_train]
            y_test = [1 if x > 5 else 0 for x in y_test]
            
            y = tensorflow.keras.utils.to_categorical(y_train)
            y_test = tensorflow.keras.utils.to_categorical(y_test)

            callbacks_list = tf.keras.callbacks.EarlyStopping(monitor='val_f1_score',
                                                              patience=20, verbose=1, mode='max', 
                                                              restore_best_weights=True)

            class_wgt = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
            wgt = {0:round(class_wgt[0], 2), 1: round(class_wgt[1], 2)}
#             wgt = {0:round(class_wgt[0], 2), 1: round(class_wgt[1], 2), 2: round(class_wgt[2], 2)}

            model = mega_model(input_shape=[(2560, 1), (2560, 3)], 
                               attx_type=conn_type,
                               attx_st=conn_stage,
                               classes = num_classes)
            mod_1 = inspect.getsource(mega_model)
            model.compile(optimizer=opt, loss=focal_loss_fx(), metrics=['acc', tfa.metrics.F1Score(num_classes=num_classes, threshold=0.5, average = 'macro')])
            print('Testing on {}'.format(i))

            hist = model.fit([X_ecg, X_eda], y, epochs=300, verbose=2, shuffle=True,
                            batch_size = 256, validation_data = ([X_test_ecg, X_test_eda], y_test),
                            callbacks=[tb, callbacks_list]) # , class_weight=wgt
            y_pred_i = model.predict([X_test_ecg, X_test_eda], batch_size = 128)

            pred_list = list()
            test_y = list()

            for n in range(len(y_pred_i)):
                pred_list.append(np.argmax(y_pred_i[n]))
                test_y.append(np.argmax(y_test[n]))

            gc.collect()

            print(classification_report(pred_list, test_y))
            a = classification_report(pred_list, test_y,
                                      target_names = ['Baseline', 'Stress'],
                                      output_dict=True)

            clr[i] = a
            hs[i] = hist

            roc_auc = roc_auc_score(y_test.astype('int'), y_pred_i, multi_class='ovo', average='weighted')
            scores = {'roc_auc': roc_auc, 'pred_prob': y_pred_i,
                        'pred': pred_list, 'test_cat': y_test, 'test': test_y}

            model.save(os.path.join(model_arch, 'model_{}'.format(i)))
            model_wgt_path = os.path.join(model_weights, '_model_{}'.format(i))
            model.save_weights(os.path.join(model_wgt_path, 'model_{}'.format(i)))

            with open(os.path.join(model_report, 'Test_fold_{}_report.pickle'.format(i)), 'wb') as handle:
                pickle.dump(clr, handle, protocol= pickle.HIGHEST_PROTOCOL)

            with open(os.path.join(model_data, 'Test_fold_{}_data.pickle'.format(i)), 'wb') as handle:
                pickle.dump(hist.history, handle, protocol=pickle.HIGHEST_PROTOCOL)

            with open(os.path.join(model_score, 'Test_fold_{}_scores.pickle'.format(i)), 'wb') as handle:
                pickle.dump(scores, handle, protocol=pickle.HIGHEST_PROTOCOL)

            create_csv(model_files, a, method, mod_1, dataset_name=dataset_name)
            K.clear_session()
            
        print("--------------------------------------------------------------------------")
        print('Classfication report for Type {}, Stage {}'.format(conn_type, conn_stage))    
        score_class(clr)
        print("--------------------------------------------------------------------------")

(None, 2560, 1)
(None, 2560, 3)
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 2560, 1)]    0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 2560, 3)]    0                                            
__________________________________________________________________________________________________
conv_stage1_ecga (Conv1D)       (None, 2560, 32)     2080        input_1[0][0]                    
__________________________________________________________________________________________________
conv_stage1_edaa (Conv1D)       (None, 2560, 32)     6176        input_2[0][0]                    
______________________________________________________________

ValueError: in user code:

    c:\Users\Anubhav\anaconda3\envs\acii\lib\site-packages\tensorflow\python\keras\engine\training.py:1233 test_function  *
        return step_function(self, iterator)
    c:\Users\Anubhav\anaconda3\envs\acii\lib\site-packages\tensorflow_addons\metrics\f_scores.py:166 update_state  *
        self.weights_intermediate.assign_add(_weighted_sum(y_true, sample_weight))
    c:\Users\Anubhav\anaconda3\envs\acii\lib\site-packages\tensorflow\python\ops\resource_variable_ops.py:844 assign_add  **
        assign_add_op = gen_resource_variable_ops.assign_add_variable_op(
    c:\Users\Anubhav\anaconda3\envs\acii\lib\site-packages\tensorflow\python\ops\gen_resource_variable_ops.py:55 assign_add_variable_op
        _, _, _op, _outputs = _op_def_library._apply_op_helper(
    c:\Users\Anubhav\anaconda3\envs\acii\lib\site-packages\tensorflow\python\framework\op_def_library.py:748 _apply_op_helper
        op = g._create_op_internal(op_type_name, inputs, dtypes=None,
    c:\Us

In [80]:
num_classes = 2
opt = Adam(learning_rate = 0.001)
model = mega_model(input_shape=[(2560, 1), (2560, 3)], attx_type='III', attx_st='all', classes = num_classes)

model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['acc', tfa.metrics.F1Score(num_classes=num_classes, threshold=0.5, average = 'macro')])

print(model.summary())

method = 'LOSO'
dataset_name = 'cola'

attx_type = ['II']
attx_st = ['two_three', 'all']

# attx_type = ['III']
# attx_st = ['all']

for conn_type in attx_type:
    
    for conn_stage in attx_st:
        print("--------------------------------------------------------------------------")
        print("Training for Type {}, Stage {}".format(conn_type, conn_stage))
        print("--------------------------------------------------------------------------\n")        
        
        hs, preds, clr = {}, {}, {}

        path_logs = r'X:/Data Files/TAFFC/Cola/'
        tensorbrd_dir, model_report, model_data, model_score, model_arch, model_fid, model_weights, model_files = create_dirs(path_logs)

        for i in ['1818', '1892', '1929', '1933']:
            opt = tf.keras.optimizers.Adadelta(learning_rate = 0.001, rho=0.95)
            tb = tensorflow.keras.callbacks.TensorBoard(log_dir = os.path.join(tensorbrd_dir,
                                                                               datetime.datetime.now().strftime("%Y%m%d-%H%M%S")))

            X_test_ecg = sub_dict_ecg[i]
            y_test = sub_label_ecg[i]
            X_test_eda = sub_dict_eda[i]

            X_test_ecg = vstack(X_test_ecg)
            X_test_eda = vstack(X_test_eda)
            y_test = [x for z in y_test for x in z]


            X_ecg = [vstack(v) for k, v in sub_dict_ecg.items() if k != i]
            X_eda = [vstack(v) for k, v in sub_dict_eda.items() if k != i]
            y_train = [hstack(np.asarray(v)) for k, v in sub_label_ecg.items() if k != i]

            X_ecg = vstack(X_ecg)
            X_eda = vstack(X_eda)
            y_train = hstack(np.asarray(y_train))

            y_train = [1 if x > 5 else 0 for x in y_train]
            y_test = [1 if x > 5 else 0 for x in y_test]
            
            y = tensorflow.keras.utils.to_categorical(y_train)
            y_test = tensorflow.keras.utils.to_categorical(y_test)

            callbacks_list = tf.keras.callbacks.EarlyStopping(monitor='val_f1_score',
                                                              patience=20, verbose=1, mode='max', 
                                                              restore_best_weights=True)

            class_wgt = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
            wgt = {0:round(class_wgt[0], 2), 1: round(class_wgt[1], 2)}
#             wgt = {0:round(class_wgt[0], 2), 1: round(class_wgt[1], 2), 2: round(class_wgt[2], 2)}

            model = mega_model(input_shape=[(2560, 1), (2560, 3)], 
                               attx_type=conn_type,
                               attx_st=conn_stage,
                               classes = num_classes)
            mod_1 = inspect.getsource(mega_model)
            model.compile(optimizer=opt, loss=focal_loss_fx(), metrics=['acc', tfa.metrics.F1Score(num_classes=num_classes, threshold=0.5, average = 'macro')])
            print('Testing on {}'.format(i))

            hist = model.fit([X_ecg, X_eda], y, epochs=300, verbose=2, shuffle=True,
                            batch_size = 256, validation_data = ([X_test_ecg, X_test_eda], y_test),
                            callbacks=[tb, callbacks_list]) # , class_weight=wgt
            y_pred_i = model.predict([X_test_ecg, X_test_eda], batch_size = 128)

            pred_list = list()
            test_y = list()

            for n in range(len(y_pred_i)):
                pred_list.append(np.argmax(y_pred_i[n]))
                test_y.append(np.argmax(y_test[n]))

            gc.collect()

            print(classification_report(pred_list, test_y))
            a = classification_report(pred_list, test_y,
                                      target_names = ['Baseline', 'Stress'],
                                      output_dict=True)

            clr[i] = a
            hs[i] = hist

            roc_auc = roc_auc_score(y_test.astype('int'), y_pred_i, multi_class='ovo', average='weighted')
            scores = {'roc_auc': roc_auc, 'pred_prob': y_pred_i,
                        'pred': pred_list, 'test_cat': y_test, 'test': test_y}

            model.save(os.path.join(model_arch, 'model_{}'.format(i)))
            model_wgt_path = os.path.join(model_weights, '_model_{}'.format(i))
            model.save_weights(os.path.join(model_wgt_path, 'model_{}'.format(i)))

            with open(os.path.join(model_report, 'Test_fold_{}_report.pickle'.format(i)), 'wb') as handle:
                pickle.dump(clr, handle, protocol= pickle.HIGHEST_PROTOCOL)

            with open(os.path.join(model_data, 'Test_fold_{}_data.pickle'.format(i)), 'wb') as handle:
                pickle.dump(hist.history, handle, protocol=pickle.HIGHEST_PROTOCOL)

            with open(os.path.join(model_score, 'Test_fold_{}_scores.pickle'.format(i)), 'wb') as handle:
                pickle.dump(scores, handle, protocol=pickle.HIGHEST_PROTOCOL)

            create_csv(model_files, a, method, mod_1, dataset_name=dataset_name)
            K.clear_session()
            
        print("--------------------------------------------------------------------------")
        print('Classfication report for Type {}, Stage {}'.format(conn_type, conn_stage))    
        score_class(clr)
        print("--------------------------------------------------------------------------")

(None, 2560, 1)
(None, 2560, 3)
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 2560, 1)]    0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 2560, 3)]    0                                            
__________________________________________________________________________________________________
conv_stage1_ecga (Conv1D)       (None, 2560, 32)     2080        input_3[0][0]                    
__________________________________________________________________________________________________
conv_stage1_edaa (Conv1D)       (None, 2560, 32)     6176        input_4[0][0]                    
____________________________________________________________

In [79]:
y_test.shape

(351, 1)

In [2]:
num_classes = 2
opt = Adam(learning_rate = 0.001)
model = mega_model(input_shape=[(2560, 1), (2560, 3)], attx_type='III', attx_st='all', classes = num_classes)

model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['acc', tfa.metrics.F1Score(num_classes=num_classes, threshold=0.5, average = 'macro')])

print(model.summary())

method = 'LOSO'
dataset_name = 'cola'

attx_type = ['II']
attx_st = ['two']

# attx_type = ['III']
# attx_st = ['all']

for conn_type in attx_type:
    
    for conn_stage in attx_st:
        print("--------------------------------------------------------------------------")
        print("Training for Type {}, Stage {}".format(conn_type, conn_stage))
        print("--------------------------------------------------------------------------\n")        
        
        hs, preds, clr = {}, {}, {}

        path_logs = r'X:/Data Files/TAFFC/Cola/'
        tensorbrd_dir, model_report, model_data, model_score, model_arch, model_fid, model_weights, model_files = create_dirs(path_logs)

        for i in sub_dict_ecg.keys():

            if i in ['1765']:
                continue

            opt = tf.keras.optimizers.Adadelta(learning_rate = 0.001, rho=0.95)
            tb = tensorflow.keras.callbacks.TensorBoard(log_dir = os.path.join(tensorbrd_dir,
                                                                               datetime.datetime.now().strftime("%Y%m%d-%H%M%S")))

            X_test_ecg = sub_dict_ecg[i]
            y_test = sub_label_ecg[i]
            X_test_eda = sub_dict_eda[i]

            X_test_ecg = vstack(X_test_ecg)
            X_test_eda = vstack(X_test_eda)
            y_test = [x for z in y_test for x in z]


            X_ecg = [vstack(v) for k, v in sub_dict_ecg.items() if k != i]
            X_eda = [vstack(v) for k, v in sub_dict_eda.items() if k != i]
            y_train = [hstack(np.asarray(v)) for k, v in sub_label_ecg.items() if k != i]

            X_ecg = vstack(X_ecg)
            X_eda = vstack(X_eda)
            y_train = hstack(np.asarray(y_train))

            y_train = [1 if x > 5 else 0 for x in y_train]
            y_test = [1 if x > 5 else 0 for x in y_test]
            
            y = tensorflow.keras.utils.to_categorical(y_train)
            y_test = tensorflow.keras.utils.to_categorical(y_test)

            callbacks_list = tf.keras.callbacks.EarlyStopping(monitor='val_f1_score',
                                                              patience=20, verbose=1, mode='max', 
                                                              restore_best_weights=True)

            class_wgt = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
            wgt = {0:round(class_wgt[0], 2), 1: round(class_wgt[1], 2)}
#             wgt = {0:round(class_wgt[0], 2), 1: round(class_wgt[1], 2), 2: round(class_wgt[2], 2)}

            model = mega_model(input_shape=[(2560, 1), (2560, 3)], 
                               attx_type=conn_type,
                               attx_st=conn_stage,
                               classes = num_classes)
            mod_1 = inspect.getsource(mega_model)
            model.compile(optimizer=opt, loss=focal_loss_fx(), metrics=['acc', tfa.metrics.F1Score(num_classes=num_classes, threshold=0.5, average = 'macro')])
            print('Testing on {}'.format(i))

            hist = model.fit([X_ecg, X_eda], y, epochs=300, verbose=2, shuffle=True,
                            batch_size = 256, validation_data = ([X_test_ecg, X_test_eda], y_test),
                            callbacks=[tb, callbacks_list]) # , class_weight=wgt
            y_pred_i = model.predict([X_test_ecg, X_test_eda], batch_size = 128)

            pred_list = list()
            test_y = list()

            for n in range(len(y_pred_i)):
                pred_list.append(np.argmax(y_pred_i[n]))
                test_y.append(np.argmax(y_test[n]))

            gc.collect()

            print(classification_report(pred_list, test_y))
            a = classification_report(pred_list, test_y,
                                      target_names = ['Baseline', 'Stress'],
                                      output_dict=True)

            clr[i] = a
            hs[i] = hist

            roc_auc = roc_auc_score(y_test.astype('int'), y_pred_i, multi_class='ovo', average='weighted')
            scores = {'roc_auc': roc_auc, 'pred_prob': y_pred_i,
                        'pred': pred_list, 'test_cat': y_test, 'test': test_y}

            model.save(os.path.join(model_arch, 'model_{}'.format(i)))
            model_wgt_path = os.path.join(model_weights, '_model_{}'.format(i))
            model.save_weights(os.path.join(model_wgt_path, 'model_{}'.format(i)))

            with open(os.path.join(model_report, 'Test_fold_{}_report.pickle'.format(i)), 'wb') as handle:
                pickle.dump(clr, handle, protocol= pickle.HIGHEST_PROTOCOL)

            with open(os.path.join(model_data, 'Test_fold_{}_data.pickle'.format(i)), 'wb') as handle:
                pickle.dump(hist.history, handle, protocol=pickle.HIGHEST_PROTOCOL)

            with open(os.path.join(model_score, 'Test_fold_{}_scores.pickle'.format(i)), 'wb') as handle:
                pickle.dump(scores, handle, protocol=pickle.HIGHEST_PROTOCOL)

            create_csv(model_files, a, method, mod_1, dataset_name=dataset_name)
            K.clear_session()
            
        print("--------------------------------------------------------------------------")
        print('Classfication report for Type {}, Stage {}'.format(conn_type, conn_stage))
        score_class(clr)
        print("--------------------------------------------------------------------------")

(None, 2560, 1)
(None, 2560, 3)
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 2560, 1)]    0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 2560, 3)]    0                                            
__________________________________________________________________________________________________
conv_stage1_ecga (Conv1D)       (None, 2560, 32)     2080        input_1[0][0]                    
__________________________________________________________________________________________________
conv_stage1_edaa (Conv1D)       (None, 2560, 32)     6176        input_2[0][0]                    
______________________________________________________________

In [81]:
num_classes = 2
opt = Adam(learning_rate = 0.001)
model = mega_model(input_shape=[(2560, 1), (2560, 3)], attx_type='III', attx_st='all', classes = num_classes)

model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['acc', tfa.metrics.F1Score(num_classes=num_classes, threshold=0.5, average = 'macro')])

print(model.summary())

method = 'LOSO'
dataset_name = 'cola'

attx_type = ['FF']
attx_st = ['FF']

# attx_type = ['III']
# attx_st = ['all']

for conn_type in attx_type:
    
    for conn_stage in attx_st:
        print("--------------------------------------------------------------------------")
        print("Training for Type {}, Stage {}".format(conn_type, conn_stage))
        print("--------------------------------------------------------------------------\n")        
        
        hs, preds, clr = {}, {}, {}

        path_logs = r'X:/Data Files/TAFFC/Cola/'
        tensorbrd_dir, model_report, model_data, model_score, model_arch, model_fid, model_weights, model_files = create_dirs(path_logs)

        for i in sub_dict_ecg.keys():

            if i in ['1765']:
                continue

            opt = tf.keras.optimizers.Adadelta(learning_rate = 0.001, rho=0.95)
            tb = tensorflow.keras.callbacks.TensorBoard(log_dir = os.path.join(tensorbrd_dir,
                                                                               datetime.datetime.now().strftime("%Y%m%d-%H%M%S")))

            X_test_ecg = sub_dict_ecg[i]
            y_test = sub_label_ecg[i]
            X_test_eda = sub_dict_eda[i]

            X_test_ecg = vstack(X_test_ecg)
            X_test_eda = vstack(X_test_eda)
            y_test = [x for z in y_test for x in z]


            X_ecg = [vstack(v) for k, v in sub_dict_ecg.items() if k != i]
            X_eda = [vstack(v) for k, v in sub_dict_eda.items() if k != i]
            y_train = [hstack(np.asarray(v)) for k, v in sub_label_ecg.items() if k != i]

            X_ecg = vstack(X_ecg)
            X_eda = vstack(X_eda)
            y_train = hstack(np.asarray(y_train))

            y_train = [1 if x > 5 else 0 for x in y_train]
            y_test = [1 if x > 5 else 0 for x in y_test]
            
            y = tensorflow.keras.utils.to_categorical(y_train)
            y_test = tensorflow.keras.utils.to_categorical(y_test)

            callbacks_list = tf.keras.callbacks.EarlyStopping(monitor='val_f1_score',
                                                              patience=20, verbose=1, mode='max', 
                                                              restore_best_weights=True)

            class_wgt = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
            wgt = {0:round(class_wgt[0], 2), 1: round(class_wgt[1], 2)}
#             wgt = {0:round(class_wgt[0], 2), 1: round(class_wgt[1], 2), 2: round(class_wgt[2], 2)}

            model = mega_model(input_shape=[(2560, 1), (2560, 3)], 
                               attx_type=conn_type,
                               attx_st=conn_stage,
                               classes = num_classes)
            mod_1 = inspect.getsource(mega_model)
            model.compile(optimizer=opt, loss=focal_loss_fx(), metrics=['acc', tfa.metrics.F1Score(num_classes=num_classes, threshold=0.5, average = 'macro')])
            print('Testing on {}'.format(i))

            hist = model.fit([X_ecg, X_eda], y, epochs=300, verbose=2, shuffle=True,
                            batch_size = 256, validation_data = ([X_test_ecg, X_test_eda], y_test),
                            callbacks=[tb, callbacks_list]) # , class_weight=wgt
            y_pred_i = model.predict([X_test_ecg, X_test_eda], batch_size = 128)

            pred_list = list()
            test_y = list()

            for n in range(len(y_pred_i)):
                pred_list.append(np.argmax(y_pred_i[n]))
                test_y.append(np.argmax(y_test[n]))

            gc.collect()

            print(classification_report(pred_list, test_y))
            a = classification_report(pred_list, test_y,
                                      target_names = ['Baseline', 'Stress'],
                                      output_dict=True)

            clr[i] = a
            hs[i] = hist

            roc_auc = roc_auc_score(y_test.astype('int'), y_pred_i, multi_class='ovo', average='weighted')
            scores = {'roc_auc': roc_auc, 'pred_prob': y_pred_i,
                        'pred': pred_list, 'test_cat': y_test, 'test': test_y}

            model.save(os.path.join(model_arch, 'model_{}'.format(i)))
            model_wgt_path = os.path.join(model_weights, '_model_{}'.format(i))
            model.save_weights(os.path.join(model_wgt_path, 'model_{}'.format(i)))

            with open(os.path.join(model_report, 'Test_fold_{}_report.pickle'.format(i)), 'wb') as handle:
                pickle.dump(clr, handle, protocol= pickle.HIGHEST_PROTOCOL)

            with open(os.path.join(model_data, 'Test_fold_{}_data.pickle'.format(i)), 'wb') as handle:
                pickle.dump(hist.history, handle, protocol=pickle.HIGHEST_PROTOCOL)

            with open(os.path.join(model_score, 'Test_fold_{}_scores.pickle'.format(i)), 'wb') as handle:
                pickle.dump(scores, handle, protocol=pickle.HIGHEST_PROTOCOL)

            create_csv(model_files, a, method, mod_1, dataset_name=dataset_name)
            K.clear_session()
            
        print("--------------------------------------------------------------------------")
        print('Classfication report for Type {}, Stage {}'.format(conn_type, conn_stage))    
        score_class(clr)
        print("--------------------------------------------------------------------------")

(None, 2560, 1)
(None, 2560, 3)
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 2560, 1)]    0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 2560, 3)]    0                                            
__________________________________________________________________________________________________
conv_stage1_ecga (Conv1D)       (None, 2560, 32)     2080        input_1[0][0]                    
__________________________________________________________________________________________________
conv_stage1_edaa (Conv1D)       (None, 2560, 32)     6176        input_2[0][0]                    
______________________________________________________________

# ECG and EDA VGG

In [3]:
num_classes = 2
# opt = Adam(learning_rate = 0.001)
# model = mega_model(input_shape=[(2560, 1), (2560, 3)], attx_type='III', attx_st='all', classes = num_classes)

# model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['acc', tfa.metrics.F1Score(num_classes=num_classes, threshold=0.5, average = 'macro')])

# print(model.summary())

method = 'LOSO'
dataset_name = 'cola'

attx_type = ['FF']
attx_st = ['FF']
modType = ['ecg', 'eda']


for mod in modType:

    main_path = r"X:\Thesis\matb2\Processed_Data"
    with open(os.path.join(main_path, 'cola_labels.pickle'), 'rb') as handle:
        sub_label = pickle.load(handle)

    if mod == 'ecg':
        ipShape = [(2560, 1)]
        with open(os.path.join(main_path, 'cola_ecg.pickle'), 'rb') as handle:
            sub_dict = pickle.load(handle)
    else:
        ipShape = [(2560, 3)]
        with open(os.path.join(main_path, 'cola_eda.pickle'), 'rb') as handle:
            sub_dict = pickle.load(handle)

    print("--------------------------------------------------------------------------")
    print("Training for Modality {}".format(mod))
    print("--------------------------------------------------------------------------\n")        
    
    hs, preds, clr = {}, {}, {}

    path_logs = r'X:/Data Files/TAFFC/Cola/'
    tensorbrd_dir, model_report, model_data, model_score, model_arch, model_fid, model_weights, model_files = create_dirs(path_logs)

    for i in sub_dict.keys():

        if i in ['1765']:
            continue

        opt = tf.keras.optimizers.Adadelta(learning_rate = 0.001, rho=0.95)
        tb = tensorflow.keras.callbacks.TensorBoard(log_dir = os.path.join(tensorbrd_dir,
                                                                        datetime.datetime.now().strftime("%Y%m%d-%H%M%S")))

        X_test = sub_dict[i]
        y_test = sub_label[i]

        X_test = vstack(X_test)
        y_test = [x for z in y_test for x in z]


        X = [vstack(v) for k, v in sub_dict.items() if k != i]
        y_train = [hstack(np.asarray(v)) for k, v in sub_label.items() if k != i]

        X = vstack(X)
        y_train = hstack(np.asarray(y_train))

        y_train = [1 if x > 5 else 0 for x in y_train]
        y_test = [1 if x > 5 else 0 for x in y_test]
        
        y = tensorflow.keras.utils.to_categorical(y_train)
        y_test = tensorflow.keras.utils.to_categorical(y_test)

        callbacks_list = tf.keras.callbacks.EarlyStopping(monitor='val_f1_score',
                                                        patience=20, verbose=1, mode='max', 
                                                        restore_best_weights=True)

        class_wgt = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
        wgt = {0:round(class_wgt[0], 2), 1: round(class_wgt[1], 2)}
#             wgt = {0:round(class_wgt[0], 2), 1: round(class_wgt[1], 2), 2: round(class_wgt[2], 2)}

        model = mega_model_ecg(input_shape=ipShape, classes = num_classes)
        mod_1 = inspect.getsource(mega_model_ecg)

        model.compile(optimizer=opt, loss=focal_loss_fx(), metrics=['acc', tfa.metrics.F1Score(num_classes=num_classes, threshold=0.5, average = 'macro')])
        print('Testing on {}'.format(i))

        hist = model.fit([X], y, epochs=300, verbose=2, shuffle=True,
                        batch_size = 256, validation_data = ([X_test], y_test),
                        callbacks=[tb, callbacks_list]) # , class_weight=wgt
        y_pred_i = model.predict([X_test], batch_size = 128)

        pred_list = list()
        test_y = list()

        for n in range(len(y_pred_i)):
            pred_list.append(np.argmax(y_pred_i[n]))
            test_y.append(np.argmax(y_test[n]))

        gc.collect()

        print(classification_report(pred_list, test_y))
        a = classification_report(pred_list, test_y,
                                target_names = ['Baseline', 'Stress'],
                                output_dict=True)

        clr[i] = a
        hs[i] = hist

        roc_auc = roc_auc_score(y_test.astype('int'), y_pred_i, multi_class='ovo', average='weighted')
        scores = {'roc_auc': roc_auc, 'pred_prob': y_pred_i,
                    'pred': pred_list, 'test_cat': y_test, 'test': test_y}

        model.save(os.path.join(model_arch, 'model_{}'.format(i)))
        model_wgt_path = os.path.join(model_weights, '_model_{}'.format(i))
        model.save_weights(os.path.join(model_wgt_path, 'model_{}'.format(i)))

        with open(os.path.join(model_report, 'Test_fold_{}_report.pickle'.format(i)), 'wb') as handle:
            pickle.dump(clr, handle, protocol= pickle.HIGHEST_PROTOCOL)

        with open(os.path.join(model_data, 'Test_fold_{}_data.pickle'.format(i)), 'wb') as handle:
            pickle.dump(hist.history, handle, protocol=pickle.HIGHEST_PROTOCOL)

        with open(os.path.join(model_score, 'Test_fold_{}_scores.pickle'.format(i)), 'wb') as handle:
            pickle.dump(scores, handle, protocol=pickle.HIGHEST_PROTOCOL)

        create_csv(model_files, a, method, mod_1, dataset_name=dataset_name)
        K.clear_session()
        
    print("--------------------------------------------------------------------------")
    print('Classfication report for Ablation {}'.format(mod))    
    score_class(clr)
    print("--------------------------------------------------------------------------")

--------------------------------------------------------------------------
Training for Modality ecg
--------------------------------------------------------------------------

Model files saved in:  X:/Data Files/TAFFC/Cola/experiment_20220607-132454
Tensorboard files for model saved in:  X:/Data Files/TAFFC/Cola/experiment_20220607-132454\t_b
Model report files saved in :  X:/Data Files/TAFFC/Cola/experiment_20220607-132454\reports
Model weights saved in :  X:/Data Files/TAFFC/Cola/experiment_20220607-132454\model_weights
(None, 2560, 1)
Testing on 1105
Epoch 1/300
27/27 - 30s - loss: 0.6947 - acc: 0.4689 - f1_score: 0.3192 - val_loss: 0.6952 - val_acc: 0.4167 - val_f1_score: 0.2941
Epoch 2/300
27/27 - 4s - loss: 0.6943 - acc: 0.4686 - f1_score: 0.3193 - val_loss: 0.6947 - val_acc: 0.4167 - val_f1_score: 0.2941
Epoch 3/300
27/27 - 4s - loss: 0.6940 - acc: 0.4663 - f1_score: 0.3281 - val_loss: 0.6942 - val_acc: 0.4167 - val_f1_score: 0.3001
Epoch 4/300
27/27 - 3s - loss: 0.6937 - acc:

In [ ]:
ecg = 'experiment_20220607-132454'
eda = 'experiment_20220607-134518'




# RESNET

In [6]:
num_classes = 2
opt = Adam(learning_rate = 0.001)
model = mega_resnet(input_shape=[(2560, 1), (2560, 3)], attx_type='II', attx_st='two_three', classes = num_classes)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['acc', tfa.metrics.F1Score(num_classes=num_classes, threshold=0.5, average = 'macro')])
print(model.summary())

method = 'LOSO'
dataset_name = 'cola'

attx_type = ['II']
attx_st = ['two_three']

# attx_type = ['III']
# attx_st = ['all']

for conn_type in attx_type:
    
    for conn_stage in attx_st:
        print("--------------------------------------------------------------------------")
        print("Training for Type {}, Stage {}".format(conn_type, conn_stage))
        print("--------------------------------------------------------------------------\n")        
        
        hs, preds, clr = {}, {}, {}

        path_logs = r'X:/Data Files/TAFFC/Cola/'
        tensorbrd_dir, model_report, model_data, model_score, model_arch, model_fid, model_weights, model_files = create_dirs(path_logs)

        for i in sub_dict_ecg.keys():

            if i in ['1765']:
                continue

            opt = tf.keras.optimizers.Adadelta(learning_rate = 0.001, rho=0.95)
            tb = tensorflow.keras.callbacks.TensorBoard(log_dir = os.path.join(tensorbrd_dir,
                                                                               datetime.datetime.now().strftime("%Y%m%d-%H%M%S")))

            X_test_ecg = sub_dict_ecg[i]
            y_test = sub_label_ecg[i]
            X_test_eda = sub_dict_eda[i]

            X_test_ecg = vstack(X_test_ecg)
            X_test_eda = vstack(X_test_eda)
            y_test = [x for z in y_test for x in z]

            X_ecg = [vstack(v) for k, v in sub_dict_ecg.items() if k != i]
            X_eda = [vstack(v) for k, v in sub_dict_eda.items() if k != i]
            y_train = [hstack(np.asarray(v)) for k, v in sub_label_ecg.items() if k != i]

            X_ecg = vstack(X_ecg)
            X_eda = vstack(X_eda)
            y_train = hstack(np.asarray(y_train))

            y_train = [1 if x > 5 else 0 for x in y_train]
            y_test = [1 if x > 5 else 0 for x in y_test]
            
            y = tensorflow.keras.utils.to_categorical(y_train)
            y_test = tensorflow.keras.utils.to_categorical(y_test)

            callbacks_list = tf.keras.callbacks.EarlyStopping(monitor='val_f1_score',
                                                              patience=20, verbose=1, mode='max', 
                                                              restore_best_weights=True)

            class_wgt = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
            wgt = {0:round(class_wgt[0], 2), 1: round(class_wgt[1], 2)}
#             wgt = {0:round(class_wgt[0], 2), 1: round(class_wgt[1], 2), 2: round(class_wgt[2], 2)}

            model = mega_resnet(input_shape=[(2560, 1), (2560, 3)], 
                               attx_type=conn_type,
                               attx_st=conn_stage,
                               classes = num_classes)
            mod_1 = inspect.getsource(mega_model)
            model.compile(optimizer=opt, loss=focal_loss_fx(), metrics=['acc', tfa.metrics.F1Score(num_classes=num_classes, threshold=0.5, average = 'macro')])
            print('Testing on {}'.format(i))

            hist = model.fit([X_ecg, X_eda], y, epochs=300, verbose=2, shuffle=True,
                            batch_size = 256, validation_data = ([X_test_ecg, X_test_eda], y_test),
                            callbacks=[tb, callbacks_list]) # , class_weight=wgt
            y_pred_i = model.predict([X_test_ecg, X_test_eda], batch_size = 128)

            pred_list = list()
            test_y = list()

            for n in range(len(y_pred_i)):
                pred_list.append(np.argmax(y_pred_i[n]))
                test_y.append(np.argmax(y_test[n]))

            gc.collect()

            print(classification_report(pred_list, test_y))
            a = classification_report(pred_list, test_y,
                                      target_names = ['Baseline', 'Stress'],
                                      output_dict=True)

            clr[i] = a
            hs[i] = hist

            roc_auc = roc_auc_score(y_test.astype('int'), y_pred_i, multi_class='ovo', average='weighted')
            scores = {'roc_auc': roc_auc, 'pred_prob': y_pred_i,
                        'pred': pred_list, 'test_cat': y_test, 'test': test_y}

            model.save(os.path.join(model_arch, 'model_{}'.format(i)))
            model_wgt_path = os.path.join(model_weights, '_model_{}'.format(i))
            model.save_weights(os.path.join(model_wgt_path, 'model_{}'.format(i)))

            with open(os.path.join(model_report, 'Test_fold_{}_report.pickle'.format(i)), 'wb') as handle:
                pickle.dump(clr, handle, protocol= pickle.HIGHEST_PROTOCOL)

            with open(os.path.join(model_data, 'Test_fold_{}_data.pickle'.format(i)), 'wb') as handle:
                pickle.dump(hist.history, handle, protocol=pickle.HIGHEST_PROTOCOL)

            with open(os.path.join(model_score, 'Test_fold_{}_scores.pickle'.format(i)), 'wb') as handle:
                pickle.dump(scores, handle, protocol=pickle.HIGHEST_PROTOCOL)

            create_csv(model_files, a, method, mod_1, dataset_name=dataset_name)
            K.clear_session()
            
        print("--------------------------------------------------------------------------")
        print('Classfication report for Type {}, Stage {}'.format(conn_type, conn_stage))    
        score_class(clr)
        print("--------------------------------------------------------------------------")

(None, 2560, 1)
(None, 2560, 3)
(None, 366, 64)
(None, 366, 64)
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 2560, 1)]    0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 2560, 3)]    0                                            
__________________________________________________________________________________________________
conv_res_stage1_ecg_a (Conv1D)  (None, 366, 32)      64          input_1[0][0]                    
__________________________________________________________________________________________________
conv_res_stage1_eda_a (Conv1D)  (None, 366, 32)      128         input_2[0][0]                    
______________________________

In [3]:
num_classes = 2
opt = Adam(learning_rate = 0.001)
model = mega_resnet(input_shape=[(2560, 1), (2560, 3)], attx_type='FF', attx_st='FF', classes = num_classes)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['acc', tfa.metrics.F1Score(num_classes=num_classes, threshold=0.5, average = 'macro')])
print(model.summary())

method = 'LOSO'
dataset_name = 'cola'

attx_type = ['FF']
attx_st = ['FF']

# attx_type = ['III']
# attx_st = ['all']

for conn_type in attx_type:
    
    for conn_stage in attx_st:
        print("--------------------------------------------------------------------------")
        print("Training for Type {}, Stage {}".format(conn_type, conn_stage))
        print("--------------------------------------------------------------------------\n")        
        
        hs, preds, clr = {}, {}, {}

        path_logs = r'X:/Data Files/TAFFC/Cola/'
        tensorbrd_dir, model_report, model_data, model_score, model_arch, model_fid, model_weights, model_files = create_dirs(path_logs)

        for i in sub_dict_ecg.keys():

            if i in ['1765']:
                continue

            opt = tf.keras.optimizers.Adadelta(learning_rate = 0.001, rho=0.95)
            tb = tensorflow.keras.callbacks.TensorBoard(log_dir = os.path.join(tensorbrd_dir,
                                                                               datetime.datetime.now().strftime("%Y%m%d-%H%M%S")))

            X_test_ecg = sub_dict_ecg[i]
            y_test = sub_label_ecg[i]
            X_test_eda = sub_dict_eda[i]

            X_test_ecg = vstack(X_test_ecg)
            X_test_eda = vstack(X_test_eda)
            y_test = [x for z in y_test for x in z]

            X_ecg = [vstack(v) for k, v in sub_dict_ecg.items() if k != i]
            X_eda = [vstack(v) for k, v in sub_dict_eda.items() if k != i]
            y_train = [hstack(np.asarray(v)) for k, v in sub_label_ecg.items() if k != i]

            X_ecg = vstack(X_ecg)
            X_eda = vstack(X_eda)
            y_train = hstack(np.asarray(y_train))

            y_train = [1 if x > 5 else 0 for x in y_train]
            y_test = [1 if x > 5 else 0 for x in y_test]
            
            y = tensorflow.keras.utils.to_categorical(y_train)
            y_test = tensorflow.keras.utils.to_categorical(y_test)

            callbacks_list = tf.keras.callbacks.EarlyStopping(monitor='val_f1_score',
                                                              patience=5, verbose=1, mode='max', 
                                                              restore_best_weights=True)

            class_wgt = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
            wgt = {0:round(class_wgt[0], 2), 1: round(class_wgt[1], 2)}
#             wgt = {0:round(class_wgt[0], 2), 1: round(class_wgt[1], 2), 2: round(class_wgt[2], 2)}

            model = mega_resnet(input_shape=[(2560, 1), (2560, 3)], 
                               attx_type=conn_type,
                               attx_st=conn_stage,
                               classes = num_classes)
            mod_1 = inspect.getsource(mega_model)
            model.compile(optimizer=opt, loss=focal_loss_fx(), metrics=['acc', tfa.metrics.F1Score(num_classes=num_classes, threshold=0.5, average = 'macro')])
            print('Testing on {}'.format(i))

            hist = model.fit([X_ecg, X_eda], y, epochs=100, verbose=2, shuffle=True,
                            batch_size = 256, validation_data = ([X_test_ecg, X_test_eda], y_test),
                            callbacks=[tb, callbacks_list]) # , class_weight=wgt
            y_pred_i = model.predict([X_test_ecg, X_test_eda], batch_size = 128)

            pred_list = list()
            test_y = list()

            for n in range(len(y_pred_i)):
                pred_list.append(np.argmax(y_pred_i[n]))
                test_y.append(np.argmax(y_test[n]))

            gc.collect()

            print(classification_report(pred_list, test_y))
            a = classification_report(pred_list, test_y,
                                      target_names = ['Baseline', 'Stress'],
                                      output_dict=True)

            clr[i] = a
            hs[i] = hist

            roc_auc = roc_auc_score(y_test.astype('int'), y_pred_i, multi_class='ovo', average='weighted')
            scores = {'roc_auc': roc_auc, 'pred_prob': y_pred_i,
                        'pred': pred_list, 'test_cat': y_test, 'test': test_y}

            model.save(os.path.join(model_arch, 'model_{}'.format(i)))
            model_wgt_path = os.path.join(model_weights, '_model_{}'.format(i))
            model.save_weights(os.path.join(model_wgt_path, 'model_{}'.format(i)))

            with open(os.path.join(model_report, 'Test_fold_{}_report.pickle'.format(i)), 'wb') as handle:
                pickle.dump(clr, handle, protocol= pickle.HIGHEST_PROTOCOL)

            with open(os.path.join(model_data, 'Test_fold_{}_data.pickle'.format(i)), 'wb') as handle:
                pickle.dump(hist.history, handle, protocol=pickle.HIGHEST_PROTOCOL)

            with open(os.path.join(model_score, 'Test_fold_{}_scores.pickle'.format(i)), 'wb') as handle:
                pickle.dump(scores, handle, protocol=pickle.HIGHEST_PROTOCOL)

            create_csv(model_files, a, method, mod_1, dataset_name=dataset_name)
            K.clear_session()
            
        print("--------------------------------------------------------------------------")
        print('Classfication report for Type {}, Stage {}'.format(conn_type, conn_stage))    
        score_class(clr)
        print("--------------------------------------------------------------------------")

(None, 2560, 1)
(None, 2560, 3)
(None, 366, 64)
(None, 366, 64)
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 2560, 1)]    0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 2560, 3)]    0                                            
__________________________________________________________________________________________________
conv_res_stage1_ecg_a (Conv1D)  (None, 366, 32)      64          input_1[0][0]                    
__________________________________________________________________________________________________
conv_res_stage1_eda_a (Conv1D)  (None, 366, 32)      128         input_2[0][0]                    
______________________________

## ECG and EDA

In [2]:
num_classes = 2
# opt = Adam(learning_rate = 0.001)
# model = mega_model(input_shape=[(2560, 1), (2560, 3)], attx_type='III', attx_st='all', classes = num_classes)

# model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['acc', tfa.metrics.F1Score(num_classes=num_classes, threshold=0.5, average = 'macro')])

# print(model.summary())

method = 'LOSO'
dataset_name = 'cola'

attx_type = ['FF']
attx_st = ['FF']
modType = ['ecg', 'eda']


for mod in modType:

    main_path = r"X:\Thesis\matb2\Processed_Data"
    with open(os.path.join(main_path, 'cola_labels.pickle'), 'rb') as handle:
        sub_label = pickle.load(handle)

    if mod == 'ecg':
        ipShape = [(2560, 1)]
        with open(os.path.join(main_path, 'cola_ecg.pickle'), 'rb') as handle:
            sub_dict = pickle.load(handle)
    else:
        ipShape = [(2560, 3)]
        with open(os.path.join(main_path, 'cola_eda.pickle'), 'rb') as handle:
            sub_dict = pickle.load(handle)

    print("--------------------------------------------------------------------------")
    print("Training for Modality {}".format(mod))
    print("--------------------------------------------------------------------------\n")        
    
    hs, preds, clr = {}, {}, {}

    path_logs = r'X:/Data Files/TAFFC/Cola/'
    tensorbrd_dir, model_report, model_data, model_score, model_arch, model_fid, model_weights, model_files = create_dirs(path_logs)

    for i in sub_dict.keys():

        if i in ['1765']:
            continue

        opt = tf.keras.optimizers.Adadelta(learning_rate = 0.001, rho=0.95)
        tb = tensorflow.keras.callbacks.TensorBoard(log_dir = os.path.join(tensorbrd_dir,
                                                                        datetime.datetime.now().strftime("%Y%m%d-%H%M%S")))

        X_test = sub_dict[i]
        y_test = sub_label[i]

        X_test = vstack(X_test)
        y_test = [x for z in y_test for x in z]


        X = [vstack(v) for k, v in sub_dict.items() if k != i]
        y_train = [hstack(np.asarray(v)) for k, v in sub_label.items() if k != i]

        X = vstack(X)
        y_train = hstack(np.asarray(y_train))

        y_train = [1 if x > 5 else 0 for x in y_train]
        y_test = [1 if x > 5 else 0 for x in y_test]
        
        y = tensorflow.keras.utils.to_categorical(y_train)
        y_test = tensorflow.keras.utils.to_categorical(y_test)

        callbacks_list = tf.keras.callbacks.EarlyStopping(monitor='val_f1_score',
                                                        patience=7, verbose=1, mode='max', 
                                                        restore_best_weights=True)

        class_wgt = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
        wgt = {0:round(class_wgt[0], 2), 1: round(class_wgt[1], 2)}
#             wgt = {0:round(class_wgt[0], 2), 1: round(class_wgt[1], 2), 2: round(class_wgt[2], 2)}

        model = ablation_model(input_shape=ipShape, classes = num_classes)
        mod_1 = inspect.getsource(mega_model_ecg)

        model.compile(optimizer=opt, loss=focal_loss_fx(), metrics=['acc', tfa.metrics.F1Score(num_classes=num_classes, threshold=0.5, average = 'macro')])
        print('Testing on {}'.format(i))

        hist = model.fit([X], y, epochs=150, verbose=2, shuffle=True,
                        batch_size = 256, validation_data = ([X_test], y_test),
                        callbacks=[tb, callbacks_list]) # , class_weight=wgt
        y_pred_i = model.predict([X_test], batch_size = 128)

        pred_list = list()
        test_y = list()

        for n in range(len(y_pred_i)):
            pred_list.append(np.argmax(y_pred_i[n]))
            test_y.append(np.argmax(y_test[n]))

        gc.collect()

        print(classification_report(pred_list, test_y))
        a = classification_report(pred_list, test_y,
                                target_names = ['Baseline', 'Stress'],
                                output_dict=True)

        clr[i] = a
        hs[i] = hist

        roc_auc = roc_auc_score(y_test.astype('int'), y_pred_i, multi_class='ovo', average='weighted')
        scores = {'roc_auc': roc_auc, 'pred_prob': y_pred_i,
                    'pred': pred_list, 'test_cat': y_test, 'test': test_y}

        model.save(os.path.join(model_arch, 'model_{}'.format(i)))
        model_wgt_path = os.path.join(model_weights, '_model_{}'.format(i))
        model.save_weights(os.path.join(model_wgt_path, 'model_{}'.format(i)))

        with open(os.path.join(model_report, 'Test_fold_{}_report.pickle'.format(i)), 'wb') as handle:
            pickle.dump(clr, handle, protocol= pickle.HIGHEST_PROTOCOL)

        with open(os.path.join(model_data, 'Test_fold_{}_data.pickle'.format(i)), 'wb') as handle:
            pickle.dump(hist.history, handle, protocol=pickle.HIGHEST_PROTOCOL)

        with open(os.path.join(model_score, 'Test_fold_{}_scores.pickle'.format(i)), 'wb') as handle:
            pickle.dump(scores, handle, protocol=pickle.HIGHEST_PROTOCOL)

        create_csv(model_files, a, method, mod_1, dataset_name=dataset_name)
        K.clear_session()
        
    print("--------------------------------------------------------------------------")
    print('Classfication report for Ablation {}'.format(mod))    
    score_class(clr)
    print("--------------------------------------------------------------------------")

--------------------------------------------------------------------------
Training for Modality ecg
--------------------------------------------------------------------------

Model files saved in:  X:/Data Files/TAFFC/Cola/experiment_20220607-185713
Tensorboard files for model saved in:  X:/Data Files/TAFFC/Cola/experiment_20220607-185713\t_b
Model report files saved in :  X:/Data Files/TAFFC/Cola/experiment_20220607-185713\reports
Model weights saved in :  X:/Data Files/TAFFC/Cola/experiment_20220607-185713\model_weights
Testing on 1105
Epoch 1/150
27/27 - 30s - loss: 7.5103 - acc: 0.4743 - f1_score: 0.3562 - val_loss: 5.0146 - val_acc: 0.4060 - val_f1_score: 0.2888
Epoch 2/150
27/27 - 3s - loss: 6.0174 - acc: 0.4908 - f1_score: 0.4581 - val_loss: 5.0131 - val_acc: 0.4231 - val_f1_score: 0.3149
Epoch 3/150
27/27 - 3s - loss: 5.6462 - acc: 0.5104 - f1_score: 0.5083 - val_loss: 4.9955 - val_acc: 0.4444 - val_f1_score: 0.4190
Epoch 4/150
27/27 - 3s - loss: 5.5148 - acc: 0.5243 - f1_sco

In [29]:
X_ecg = [vstack(v) for k, v in sub_dict_ecg.items() if k != i]


In [33]:
X_ecg[0].shape

(468, 2560, 1)

In [15]:
y_test.shape

(4, 117)

In [20]:
y = [x for z in y_test for x in z]

In [22]:
len(y)

468